In [1]:
import requests
import re
import pandas as pd
import numpy as np
from pandas import DataFrame
import json
from bs4 import BeautifulSoup
from pygeocoder import Geocoder

In [2]:
def mcd_scrape(url):
    response = requests.get(url)
    S = BeautifulSoup(response.text, 'lxml')
    mcd = []
    list_items = S.find_all('li')
    for L in list_items:
        if "Dunkin' Donuts in" in L.text:
            addr = L.text.replace("  ","")
            addr = addr.replace("\n","")
            addr = addr.split("Dunkin' Donuts in")
            mcd.append(addr)
    mcd2 = pd.DataFrame(mcd).drop(0,axis=1).values.tolist()

    mcd3 = []
    for elem in mcd2:
        mcd3.extend(elem)
    mcd4 = []
    for elem in mcd3:
        if elem != None:
            mcd4.append(elem)
    mcd5 = pd.DataFrame(mcd4)
    mcd6 = mcd5[0].str.split(' ',2, expand = True)
    mcd7 = mcd6[2].str.split(' ',1, expand = True).drop(0,axis=1)
    mcd8=mcd6[1].values.tolist()
    mcd9=mcd7.values.tolist()
    mcd9_list = []
    for elem in mcd9:
        mcd9_list.extend(elem)
    mcd10 = pd.DataFrame()
    mcd10['address'] = mcd9_list
    mcd10['zip'] = mcd8
    return(mcd10)

In [3]:
mo_mcd = mcd_scrape("https://www.menuism.com/restaurant-locations/dunkin-donuts-181624/us/mo")
mo_mcd['state'] = "MO"
mo_mcd

,address,zip,state
0,2400 Missouri 291,64057,MO
1,2400 Missouri 291,64057,MO
2,2400 Missouri 291,64057,MO
3,2400 Missouri 291,64057,MO
4,9806 W Florissant Ave,63136,MO
5,9898 Air Cargo Road,63145,MO
6,10701 Lambert International Boulevard,63145,MO
7,9806 W Florissant Ave,63136,MO
8,9898 Air Cargo Road,63145,MO
9,10701 Lambert International Boulevard,63145,MO


In [4]:
il_mcd = mcd_scrape("https://www.menuism.com/restaurant-locations/dunkin-donuts-181624/us/il")
il_mcd['state'] = "IL"
il_mcd

,address,zip,state
0,33 E Lake St,60101,IL
1,1750 W Lake St,60101-1172,IL
2,33 E Lake St,60101,IL
3,1750 W Lake St,60101-1172,IL
4,1319 East Algonquin Road,60102,IL
...,...,...,...
991,10662 S Harlem Ave,60482-1247,IL
992,6707 W 111th St,60482-1911,IL
993,11501 S Harlem Ave,60482,IL
994,10662 S Harlem Ave,60482-1247,IL


In [5]:
both = [mo_mcd,il_mcd]
both_mcd = pd.concat(both)
cleaned_mcd = pd.DataFrame(both_mcd['zip'].str.split('-').str[0])
cleaned_mcd['address']= both_mcd['address']
cleaned_mcd['state'] = both_mcd['state']
cleaned_mcd = cleaned_mcd[['address', 'state', 'zip']]
cleaned_mcd

,address,state,zip
0,2400 Missouri 291,MO,64057
1,2400 Missouri 291,MO,64057
2,2400 Missouri 291,MO,64057
3,2400 Missouri 291,MO,64057
4,9806 W Florissant Ave,MO,63136
...,...,...,...
991,10662 S Harlem Ave,IL,60482
992,6707 W 111th St,IL,60482
993,11501 S Harlem Ave,IL,60482
994,10662 S Harlem Ave,IL,60482


In [6]:
cleaned_mcd['location'] = cleaned_mcd[['address', 'state', 'zip']].agg(' '.join, axis=1)
cleaned_mcd = cleaned_mcd.drop_duplicates()
cleaned_mcd

,address,state,zip,location
0,2400 Missouri 291,MO,64057,2400 Missouri 291 MO 64057
4,9806 W Florissant Ave,MO,63136,9806 W Florissant Ave MO 63136
5,9898 Air Cargo Road,MO,63145,9898 Air Cargo Road MO 63145
6,10701 Lambert International Boulevard,MO,63145,10701 Lambert International Boulevard MO 63145
0,33 E Lake St,IL,60101,33 E Lake St IL 60101
...,...,...,...,...
986,339 S Eastwood Dr,IL,60098,339 S Eastwood Dr IL 60098
987,335 S Eastwood Dr,IL,60098,335 S Eastwood Dr IL 60098
990,11501 S Harlem Ave,IL,60482,11501 S Harlem Ave IL 60482
991,10662 S Harlem Ave,IL,60482,10662 S Harlem Ave IL 60482


In [7]:
cleaned_mcd['location'].to_csv('../dunkin_loc.csv', index = False)
check = pd.read_csv('../dunkin_loc.csv')
check

,location
0,2400 Missouri 291 MO 64057
1,9806 W Florissant Ave MO 63136
2,9898 Air Cargo Road MO 63145
3,10701 Lambert International Boulevard MO 63145
4,33 E Lake St IL 60101
...,...
481,339 S Eastwood Dr IL 60098
482,335 S Eastwood Dr IL 60098
483,11501 S Harlem Ave IL 60482
484,10662 S Harlem Ave IL 60482


In [8]:
# used https://dash.geocod.io/ to geocode data and create new csv with lat/long info
geodata = pd.read_csv('../geocoded_dunkin_loc.csv')
geodata

,location,Latitude,Longitude,Accuracy Score,Accuracy Type,Number,Street,Unit Type,Unit Number,City,State,County,Zip,Country,Source
0,2400 Missouri 291 MO 64057,39.075001,-94.379703,0.85,rooftop,2400,S M 291 Hwy,NaN,NaN,Independence,MO,Jackson County,64055,US,Jackson
1,2400 Missouri 291 MO 64057,39.075001,-94.379703,0.85,rooftop,2400,S M 291 Hwy,NaN,NaN,Independence,MO,Jackson County,64055,US,Jackson
2,2400 Missouri 291 MO 64057,39.075001,-94.379703,0.85,rooftop,2400,S M 291 Hwy,NaN,NaN,Independence,MO,Jackson County,64055,US,Jackson
3,2400 Missouri 291 MO 64057,39.075001,-94.379703,0.85,rooftop,2400,S M 291 Hwy,NaN,NaN,Independence,MO,Jackson County,64055,US,Jackson
4,9806 W Florissant Ave MO 63136,38.745792,-90.279038,1.00,rooftop,9806,W Florissant Ave,NaN,NaN,Saint Louis,MO,St. Louis County,63136,US,St Louis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,10662 S Harlem Ave IL 60482,41.697618,-87.797904,1.00,range_interpolation,10662,S Harlem Ave,NaN,NaN,Worth,IL,Cook County,60482,US,TIGER/Line® dataset from the US Census Bureau
1002,6707 W 111th St IL 60482,41.690083,-87.786212,1.00,range_interpolation,6707,W 111th St,NaN,NaN,Worth,IL,Cook County,60482,US,TIGER/Line® dataset from the US Census Bureau
1003,11501 S Harlem Ave IL 60482,41.682778,-87.797175,1.00,range_interpolation,11501,S Harlem Ave,NaN,NaN,Worth,IL,Cook County,60482,US,TIGER/Line® dataset from the US Census Bureau
1004,10662 S Harlem Ave IL 60482,41.697618,-87.797904,1.00,range_interpolation,10662,S Harlem Ave,NaN,NaN,Worth,IL,Cook County,60482,US,TIGER/Line® dataset from the US Census Bureau


In [9]:
#insert code here that drops all rows in cleaned_mcd that don't contain the relevant zip codes

zips = []
with open('../zip_list.txt', 'r') as f:
    zips.extend(f.read().split('\n'))
zips

['63001',
 '63005',
 '63006',
 '63011',
 '63017',
 '63021',
 '63022',
 '63024',
 '63025',
 '63026',
 '63031',
 '63032',
 '63033',
 '63034',
 '63038',
 '63040',
 '63042',
 '63043',
 '63044',
 '63045',
 '63074',
 '63088',
 '63099',
 '63105',
 '63114',
 '63117',
 '63119',
 '63121',
 '63122',
 '63123',
 '63124',
 '63125',
 '63126',
 '63127',
 '63128',
 '63129',
 '63130',
 '63131',
 '63132',
 '63133',
 '63134',
 '63135',
 '63136',
 '63137',
 '63138',
 '63140',
 '63141',
 '63143',
 '63144',
 '63145',
 '63146',
 '63151',
 '63167',
 '63198']

In [10]:
complete_mcd = geodata[~geodata.Zip.isin(zips)]
complete_mcd

,location,Latitude,Longitude,Accuracy Score,Accuracy Type,Number,Street,Unit Type,Unit Number,City,State,County,Zip,Country,Source
0,2400 Missouri 291 MO 64057,39.075001,-94.379703,0.85,rooftop,2400,S M 291 Hwy,NaN,NaN,Independence,MO,Jackson County,64055,US,Jackson
1,2400 Missouri 291 MO 64057,39.075001,-94.379703,0.85,rooftop,2400,S M 291 Hwy,NaN,NaN,Independence,MO,Jackson County,64055,US,Jackson
2,2400 Missouri 291 MO 64057,39.075001,-94.379703,0.85,rooftop,2400,S M 291 Hwy,NaN,NaN,Independence,MO,Jackson County,64055,US,Jackson
3,2400 Missouri 291 MO 64057,39.075001,-94.379703,0.85,rooftop,2400,S M 291 Hwy,NaN,NaN,Independence,MO,Jackson County,64055,US,Jackson
10,33 E Lake St IL 60101,41.931216,-87.987127,1.00,rooftop,33,E Lake St,NaN,NaN,Addison,IL,DuPage County,60101,US,Dupage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,10662 S Harlem Ave IL 60482,41.697618,-87.797904,1.00,range_interpolation,10662,S Harlem Ave,NaN,NaN,Worth,IL,Cook County,60482,US,TIGER/Line® dataset from the US Census Bureau
1002,6707 W 111th St IL 60482,41.690083,-87.786212,1.00,range_interpolation,6707,W 111th St,NaN,NaN,Worth,IL,Cook County,60482,US,TIGER/Line® dataset from the US Census Bureau
1003,11501 S Harlem Ave IL 60482,41.682778,-87.797175,1.00,range_interpolation,11501,S Harlem Ave,NaN,NaN,Worth,IL,Cook County,60482,US,TIGER/Line® dataset from the US Census Bureau
1004,10662 S Harlem Ave IL 60482,41.697618,-87.797904,1.00,range_interpolation,10662,S Harlem Ave,NaN,NaN,Worth,IL,Cook County,60482,US,TIGER/Line® dataset from the US Census Bureau


In [11]:
complete_mcd = complete_mcd[["location", "Accuracy Score", "Latitude", "Longitude", "City", "State", "Zip"]]
complete_mcd = complete_mcd.rename(columns={"location":"id", "Accuracy Score":"strLocation", "Latitude":"latitude", "Longitude":"longitude", "City":"city", "State":"state", "Zip":"zip"})
complete_mcd["Store"] = "Dunkin'"
complete_mcd

,id,strLocation,latitude,longitude,city,state,zip,Store
0,2400 Missouri 291 MO 64057,0.85,39.075001,-94.379703,Independence,MO,64055,Dunkin'
1,2400 Missouri 291 MO 64057,0.85,39.075001,-94.379703,Independence,MO,64055,Dunkin'
2,2400 Missouri 291 MO 64057,0.85,39.075001,-94.379703,Independence,MO,64055,Dunkin'
3,2400 Missouri 291 MO 64057,0.85,39.075001,-94.379703,Independence,MO,64055,Dunkin'
10,33 E Lake St IL 60101,1.00,41.931216,-87.987127,Addison,IL,60101,Dunkin'
...,...,...,...,...,...,...,...,...
1001,10662 S Harlem Ave IL 60482,1.00,41.697618,-87.797904,Worth,IL,60482,Dunkin'
1002,6707 W 111th St IL 60482,1.00,41.690083,-87.786212,Worth,IL,60482,Dunkin'
1003,11501 S Harlem Ave IL 60482,1.00,41.682778,-87.797175,Worth,IL,60482,Dunkin'
1004,10662 S Harlem Ave IL 60482,1.00,41.697618,-87.797904,Worth,IL,60482,Dunkin'


In [12]:
complete_mcd.to_csv('../final_dunkin_loc.csv', index = False)
check = pd.read_csv('../final_dunkin_loc.csv')
check

,id,strLocation,latitude,longitude,city,state,zip,Store
0,2400 Missouri 291 MO 64057,0.85,39.075001,-94.379703,Independence,MO,64055,Dunkin'
1,2400 Missouri 291 MO 64057,0.85,39.075001,-94.379703,Independence,MO,64055,Dunkin'
2,2400 Missouri 291 MO 64057,0.85,39.075001,-94.379703,Independence,MO,64055,Dunkin'
3,2400 Missouri 291 MO 64057,0.85,39.075001,-94.379703,Independence,MO,64055,Dunkin'
4,33 E Lake St IL 60101,1.00,41.931216,-87.987127,Addison,IL,60101,Dunkin'
...,...,...,...,...,...,...,...,...
995,10662 S Harlem Ave IL 60482,1.00,41.697618,-87.797904,Worth,IL,60482,Dunkin'
996,6707 W 111th St IL 60482,1.00,41.690083,-87.786212,Worth,IL,60482,Dunkin'
997,11501 S Harlem Ave IL 60482,1.00,41.682778,-87.797175,Worth,IL,60482,Dunkin'
998,10662 S Harlem Ave IL 60482,1.00,41.697618,-87.797904,Worth,IL,60482,Dunkin'
